# Code to generate binary masks for COCO dataset

In [4]:
from pycocotools import coco, cocoeval, _mask
from pycocotools import mask as maskUtils 
import array
import numpy as np
# import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import os
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
%matplotlib inline

In [5]:
# CATEGORY_NAMES=['bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train','truck', 'boat',
#                 'bird','cat','dog','horse','sheep','cow', 'elephant', 'bear', 'zebra','giraffe']

CATEGORY_NAMES = ['person']
ANNOTATION_FILE_VAL = '/root/Internship-Valeo/data/COCO/annotations_trainval2014/annotations/instances_val2014.json'
ANNOTATION_FILE_TRAIN = '/root/Internship-Valeo/data/COCO/annotations_trainval2014/annotations/instances_train2014.json'


coco_train = coco.COCO(ANNOTATION_FILE_TRAIN)
coco_val = coco.COCO(ANNOTATION_FILE_VAL)

imgIds_train = []
if CATEGORY_NAMES!=None:
    # iterate for each individual class in the list
    for className in CATEGORY_NAMES:
        # get all images containing given super class
        catIds_train = coco_train.getCatIds(catNms=className)
        imgIds_train += coco_train.getImgIds(catIds=catIds_train)
        
else:
    imgIds_train = coco_train.getImgIds()

# Now, filter out the repeated images    
unique_imgIds_train = []
for i in range(len(imgIds_train)):
    if imgIds_train[i] not in unique_imgIds_train:
        unique_imgIds_train.append(imgIds_train[i])

images_train = coco_train.loadImgs(unique_imgIds_train)
dataset_size = len(images_train)

print("Number of images containing the filter classes:", dataset_size)

loading annotations into memory...
Done (t=10.81s)
creating index...
index created!
loading annotations into memory...
Done (t=5.75s)
creating index...
index created!
Number of images containing the filter classes: 45174


In [6]:
imgIds_val = []
if CATEGORY_NAMES!=None:
    # iterate for each individual class in the list
    for className in CATEGORY_NAMES:
        # get all images containing given super class
        catIds_val = coco_val.getCatIds(catNms=className)
        imgIds_val += coco_val.getImgIds(catIds=catIds_val)

else:
    imgIds_val = coco_val.getImgIds()
    

    
# Now, filter out the repeated images      
unique_imgIds_val = []
for i in range(len(imgIds_val)):
    if imgIds_val[i] not in unique_imgIds_val:
        unique_imgIds_val.append(imgIds_val[i])

images_val = coco_val.loadImgs(unique_imgIds_val)
dataset_size = len(images_val)

print("Number of images containing the filter classes:", dataset_size)

Number of images containing the filter classes: 21634


In [7]:
from random import shuffle


shuffle(imgIds_train)
shuffle(imgIds_val)

# imgIds_train = imgIds_train[0:6000]
# imgIds_val = imgIds_val[0:600]

In [8]:
train_images = ["COCO_train2014_{0:012d}.jpg".format(ids) for ids in unique_imgIds_train]
val_images = ["COCO_val2014_{0:012d}.jpg".format(ids) for ids in unique_imgIds_val]

In [9]:
print(len(train_images) , len(val_images))

45174 21634


In [10]:
train_dir = '/root/Internship-Valeo/Project/data/COCO/anntrain2014'

In [13]:
if os.listdir(train_dir) is not None:
    for f in os.listdir(train_dir):
        os.remove(train_dir+'/'+f)

In [14]:
val_dir = '/root/Internship-Valeo/Project/data/COCO/annval2014'

In [15]:
if os.listdir(val_dir) is not None:
    for f in os.listdir(val_dir):
        os.remove(val_dir+'/'+f)

In [16]:
import shutil
source_dir = '/root/Internship-Valeo/data/COCO/train2014'
for f in train_images:
    shutil.copy(source_dir+'/'+f, train_dir)

In [17]:
source_dir = '/root/Internship-Valeo/data/COCO/val2014'
for f in val_images:
    shutil.copy(source_dir+'/'+f, val_dir)

In [18]:
print(len(os.listdir(train_dir)))

45174


In [19]:
from PIL import Image

In [20]:
count = 0 

catIds_train = coco_train.getCatIds(catNms=CATEGORY_NAMES) 

for ID in unique_imgIds_train:
    file_path = "/root/Internship-Valeo/Project/data/COCO/masktrain2014/COCO_train2014_{0:012d}.jpg".format(ID)
  
    sampleImgIds = coco_train.getImgIds(imgIds = [ID])
    sampleImgDict = coco_train.loadImgs(sampleImgIds[np.random.randint(0,len(sampleImgIds))])[0]

    annIds = coco_train.getAnnIds(imgIds=sampleImgDict['id'], catIds=catIds_train, iscrowd=0)
    anns = coco_train.loadAnns(annIds)


    mask = coco_train.annToMask(anns[0])
    for i in range(len(anns)):
        mask = mask | coco_train.annToMask(anns[i])
    mask = Image.fromarray(mask * 255 , mode = "L")
    mask.save(file_path)
    count = count + 1
print(count)

45174


In [21]:
# !mkdir maskval2014

In [22]:
count = 0

catIds_val = coco_val.getCatIds(catNms=CATEGORY_NAMES) 

for ID in unique_imgIds_val:
    file_path = "/root/Internship-Valeo/Project/data/COCO/maskval2014/COCO_val2014_{0:012d}.jpg".format(ID)
  
    sampleImgIds = coco_val.getImgIds(imgIds = [ID])
    sampleImgDict = coco_val.loadImgs(sampleImgIds[np.random.randint(0,len(sampleImgIds))])[0]

    annIds = coco_val.getAnnIds(imgIds=sampleImgDict['id'], catIds=catIds_val, iscrowd=0)
    anns = coco_val.loadAnns(annIds)


    mask = coco_val.annToMask(anns[0])
    for i in range(len(anns)):
        mask = mask | coco_val.annToMask(anns[i])
    mask = Image.fromarray(mask * 255 , mode = "L")
    mask.save(file_path)
    count = count + 1
print(count)

21634
